In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn import preprocessing
from sklearn import utils
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet,SGDRegressor
from sklearn.svm import LinearSVR,SVR
from sklearn.metrics import r2_score

import pandas as pd
import numpy as np
import textract
import os
import PyPDF2
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import wordninja
import numpy as np
import nltk
from nltk.corpus import stopwords
import re 
import string
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer,LancasterStemmer
from pyresparser import ResumeParser
import warnings
import os
warnings.filterwarnings("ignore")
nltk.download("stopwords")

In [ ]:
dirs = os.getcwd()

In [ ]:
test = pd.read_csv('test.csv')
test_ID_files = test['CandidateID']
train = pd.read_csv('train.csv')
train_ID_files = train['CandidateID']
train_percentage = train['Match Percentage']

In [ ]:
(train_ID_files,train_percentage)

In [ ]:
def StemWords(word):
    stemmer = PorterStemmer()
    return stemmer.stem(word)

def getPath(filetype,filename):
    
    path = f'{filetype}/{filename}.pdf'
    
    full_path = os.path.join(dirs,path)
    
    return full_path

In [ ]:
def extract_text(path):
    stopWords = set(stopwords.words('english'))
    conv_list = []
    
    text_extracted = textract.process(path)
    
    text_extracted = text_extracted.decode()
    
    text_extracted = text_extracted.strip("\r\n")
    

    
    for el in text_extracted:
        conv_list.append(el.strip())
    
    strs = "".join(conv_list)
    
    text = wordninja.split(strs)
    
    content = " ".join(text).lower()
    
    content = " ".join([word for word in content.split() if word not in stopWords])
    
    text_encoded = content.encode(encoding='ascii',errors='ignore')
                
    text_decoded = text_encoded.decode()
    
    text = re.sub("@\S+", "", content)
    text = re.sub("\$", "", text)
    text = re.sub("https?:\/\/.*[\r\n]*", "", text)
    text = re.sub("#", "", text)
    
    punctuation = set(string.punctuation)
    text = "".join([ch for ch in text if ch not in punctuation])
    
    words = text.split(" ")
    texts = []
    
    for word in words:
        texts.append(StemWords(word))
        
    return texts

In [ ]:
def read_train_pdf_files(train_ID_files, train_percentage):
    #get english stopwords from stopwords library
    resumes = []
    ids = []
    percentages = []
    
    
    for filename in os.listdir(dirs):
        if filename == "trainResumes":
            try:
                for file_name,percentage in zip(train_ID_files,train_percentage):
                    full_path = getPath(filename,file_name)
                    texts = extract_text(full_path)
                    resumes.append(" ".join(texts))
                    ids.append(file_name)
                    percentages.append(percentage)
            except:
                print('Something went wrong')
    return ids,resumes,percentages


def read_test_files(test_ID_files):
    resumes = []
    ids = []
    for filename in zip(test_ID_files):
        
        full_path = getPath('testResumes',filename)
        #the fullpath will be : C:\Users\Camara\Desktop\Machine Learning Perfect fit\testResumes/('candidate_014',).pdf
        #let's remove the (',') from the path variable
        full_path = full_path.replace("(","")
        full_path = full_path.replace("'","")
        full_path = full_path.replace(",","")
        full_path = full_path.replace(")","")
        texts = extract_text(full_path)
        ids.append(filename)
        resumes.append(texts)
    
    return ids,resumes

In [ ]:
train_id,train_resumes,train_percentages = read_train_pdf_files(train_ID_files, train_percentage)